# Using soup

In [3]:
import scraping_class
logfile = 'log.txt' ## name your log file.
connector = scraping_class.Connector(logfile)

In [4]:
from bs4 import BeautifulSoup
import requests
main_url = 'https://kurser.ku.dk/archive'
response = connector.get(main_url,'year_archives_links')
raw_html = response[0].text
# Collecting the soup
soup = BeautifulSoup(raw_html,'html.parser')

### UCPH  Courses - Step 1: Collecting links for the yearly archives

In [5]:
# Collecting all the hyperlinks from the mainpage
all_links = [a.get('href') for a in soup.find_all('a', href=True)]

# Saving all the hyperlinks which contains the 'archive'
endpoint_links = [s for s in all_links if "archive" in s]

# Removing 'archive' from the endpoint links since the word is already in the main_url
endpoint_links = [w[7:] for w in endpoint_links]
endpoint_links

['/2017-2018', '/2016-2017', '/2015-2016', '/2014-2015', '/2013-2014']

In [6]:
# Creating the right links to scrape from the yearly archives
desired_links_year = []
for i in range(0,len(endpoint_links)):
    url = main_url + endpoint_links[i]
    desired_links_year.append(url)
desired_links_year

['https://kurser.ku.dk/archive/2017-2018',
 'https://kurser.ku.dk/archive/2016-2017',
 'https://kurser.ku.dk/archive/2015-2016',
 'https://kurser.ku.dk/archive/2014-2015',
 'https://kurser.ku.dk/archive/2013-2014']

### UCPH Courses - Step 2: Collceting links for each Study Board

In [7]:
soup_2 = []
all_links_2 = []
endpoint_links_2 = []
endpoint_links_2_final = []
desired_links_study_board = []

for i in range(0,len(desired_links_year)):
    
    main_url_2 = desired_links_year[i]
    print(main_url_2)
    
    response = connector.get(main_url_2,'year_archives_links')
    raw_html_2 = response[0].text
    # Collecting the soup
    soup_2.append(BeautifulSoup(raw_html_2,'html.parser'))
    # soup_2 is a list of lists, each lists contains html code for each yearly archive.
    
    # Collecting all the hyperlinks from the mainpage
    all_links_2.append([a.get('href') for a in soup_2[i].find_all('a', href=True)])
    
    # Saving the given year, this step only needs to be run once. 
    if i == 0:
        given_year = [w[29:] for w in desired_links_year]
   
    # Saving all the hyperlinks which contains the given year, these are the links for the Study boards
    endpoint_links_2.append([s for s in all_links_2[i] if given_year[i] in s])
    
    # Removing the year from the endpoint_links_2 links since the word is already in the desired_links_year
    endpoint_links_2_final.append([b[9:] for b in endpoint_links_2[i]])
    
    # Creating the right links to scrape from the study board archives
    for k in range(0,len(endpoint_links_2_final[i])): 
        
        # Getting elements in the lists out as strings in the correct way
        str_studyboard = ''.join(endpoint_links_2_final[i][0+k:1+k])
        
        desired_links_study_board.append(desired_links_year[i] + str_studyboard)
        
soup_2;
all_links_2;
given_year;
endpoint_links_2;
endpoint_links_2_final;
desired_links_study_board;

https://kurser.ku.dk/archive/2017-2018
https://kurser.ku.dk/archive/2016-2017
https://kurser.ku.dk/archive/2015-2016
https://kurser.ku.dk/archive/2014-2015
https://kurser.ku.dk/archive/2013-2014


In [8]:
len(desired_links_study_board)

192

In [9]:
desired_links_study_board[:5]

['https://kurser.ku.dk/archive/2017-2018/888',
 'https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0010',
 'https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0021',
 'https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0044',
 'https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0009']

### UCPH Courses - Step 3: Collecting links for each course

In [10]:
desired_links_study_board_rev = []

# We have to edit til disered_links_study_board because the course url do not contain the study board url.
for q in range(0,len(desired_links_study_board)):
    desired_links_study_board_rev.append(desired_links_study_board[q][0:38])

In [11]:
soup_3 = []
all_links_3 = []
endpoint_links_3 = []
endpoint_links_3_final=[]
desired_links_courses = []

#Looping through all studyboards in a given year and collecting the course links
for i in range(0,len(desired_links_study_board)):
    main_url_3 = desired_links_study_board[i]
    print(main_url_3)
    
    response = connector.get(main_url_3,'study_board_years_archives_links')
    raw_html_3 = response[0].text
    # Collecting the soup
    soup_3.append(BeautifulSoup(raw_html_3,'html.parser'))
    print(len(soup_3)) 
        
    # Collecting all the hyperlinks from the course page from a given study board in a given year
    all_links_3.append([a.get('href') for a in soup_3[i].find_all('a', href=True)])
    
    # Saving all the hyperlinks which contains the given year, these are the links for the Study boards
    endpoint_links_3.append([s for s in all_links_3[i] if '/course/' in s])
    
    # Removing the year and studyboard from endpoint_links_3 
    endpoint_links_3_final.append([b[18:] for b in endpoint_links_3[i]]) 
    
    # Running through each study board and collecting the course urls.
    for k in range(0,len(endpoint_links_3_final[i])):
        
        # Getting elements in the lists out as strings in the correct way
        str_course = ''.join(endpoint_links_3_final[i][k])
        
        # Desired_links_courses is a list containing all course links. 
        desired_links_courses.append(desired_links_study_board_rev[i] + str_course)


https://kurser.ku.dk/archive/2017-2018/888
1
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0010
2
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0021
3
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0044
4
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0009
5
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_PB_01
6
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_SU_0031
7
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0012
8
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0033
9
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0025
10
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0019
11
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD__BI_0031
12
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0028
13
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0027
14
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0029
15
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0034
16
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0030
17
https://kurser.ku.dk/archiv

142
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0030
143
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0031
144
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_PHD_0010
145
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0001
146
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0005
147
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_MA_0014
148
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0006
149
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0016
150
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0091
151
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0003
152
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0203
153
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0014
154
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0004
155
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0023
156
https://kurser.ku.dk/archive/2013-2014/STUDYBOARD_0010
157
https://kurser.ku.dk/archive/2013-2014/STUDYBOARD_0032
158
https://kurser.ku.dk/archive/2013-2014/STUDYB

In [39]:
print(len(desired_links_courses))
print(len(set(desired_links_courses)))
#we have no dublicates - which makes sence.

20035
20035


In [17]:
desired_links_courses[0]

'https://kurser.ku.dk/archive/2017-2018/course/UCPHAUK18U'

### UCPH Courses - Collecting the course ID's

In [30]:
course_ids = []

for i in range(0,len(desired_links_courses)):
    course_ids.append(desired_links_courses[i][46:])


print(len(course_ids))

course_ids[20034]

20035


'SVEK13099U'

In [36]:
# A set object is an unordered collection of distinct hashable objects - we convert the set back to a list.
unique_course_ids = list(set(course_ids))
len(unique_course_ids)
# We see that we have some dublicates in the course_ids list. This is becuase some courses do not change ID from year to year.

11126

### UCPH Courses - Scraping each course 

In [40]:
import pandas as pd
import numpy as np
import html5lib

In [41]:
def create_df(url):
    response = requests.get(url)
    
    course_id = url.split('/')[-2]
    year = url.split('/')[-1]

    raw_html = response.text
    soup = BeautifulSoup(raw_html,'lxml')

    info = soup.find_all("dl", {'class':'dl-horizontal'})

    comp_info = pd.DataFrame()
    cleaned_id_text = []
    for i in info[0].find_all('dt'):
        cleaned_id_text.append(i.text)
    cleaned_id__attrb_text = []
    for i in info[0].find_all('dd'):
        cleaned_id__attrb_text.append(i.text)
    for i in info[1].find_all('dt'):
        cleaned_id_text.append(i.text)
    for i in info[1].find_all('dd'):
        cleaned_id__attrb_text.append(i.text)

    cleaned_id_text.append('Year')
    cleaned_id__attrb_text.append(year)

    cleaned_id_text.append('Course id')
    cleaned_id__attrb_text.append(course_id)

    work = soup.find('ul',{'class':'list-unstyled workload clearfix'})
    workfi = work.find_all('li')

    for i in range(len(workfi)):
        if i % 2 ==0:
            cleaned_id_text.append(workfi[i].text)
        else: 
            cleaned_id__attrb_text.append(workfi[i].text)


    comp_info['Id'] = cleaned_id_text
    comp_info['Attribute'] = cleaned_id__attrb_text
    return(comp_info)

In [142]:
dct = dict()

for i in range(1,20):
    url = desired_links_courses[i]
    print(desired_links_courses[i])
    dftest=create_df(url)

    # No need for a column called Id. We just use the Id column as the index. For now.
    # Also, we rename the other column ('Attribute') to the course url since we will use this as the key in the dict. 
    dftest=dftest.set_index('Id').rename(columns = {'Attribute': desired_links_courses[i]})

    # Converting the dataframe to a dictionary
    dct_temp = dict()
    dct_temp = dftest.to_dict()
    
    # Adding to dict
    dct.update(dct_temp)
    

https://kurser.ku.dk/archive/2017-2018/course/AANA17100U
https://kurser.ku.dk/archive/2017-2018/course/AANA17101U
https://kurser.ku.dk/archive/2017-2018/course/AANA17102U
https://kurser.ku.dk/archive/2017-2018/course/AANA17103U
https://kurser.ku.dk/archive/2017-2018/course/AANA17104U
https://kurser.ku.dk/archive/2017-2018/course/AANA17105U


ValueError: Length of values does not match length of index

In [140]:

print(desired_links_courses[3])
dct[desired_links_courses[3]]

https://kurser.ku.dk/archive/2017-2018/course/AANA17102U


{'Category': 'Hours',
 'Censorship form': 'No external censorship',
 'Class Instruction': '42',
 'Course code': 'AANA17102U',
 'Course id': 'course',
 'Credit': '7,5 ECTS',
 'Duration': '1 semester',
 'Exam': '37',
 'Exam period': 'The essay must be uploaded to Digital Exam no later than June\r\n7th at 12.00 (noon).',
 'Language': 'English',
 'Level': 'Bachelor\r\nBachelor choice Full Degree Master choice',
 'Marking scale': '7-point grading scale',
 'Placement': 'Spring',
 'Practical exercises': '44',
 'Preparation': '84',
 'Re-exam': '1. re-exam:An essay with a revised problem statement must be submitted at\r\nthe announced date. The students must sign up for the 1.\r\nre-exam.2. re-exam:A new essay with a revised problem statement must be submitted\r\nat the announced date next semester. The students must sign up for\r\nthe 2. re-exam.',
 'Schedule': 'more information to come.',
 'Study board': 'Department of Anthropology, Study Council',
 'Total': '207',
 'Type of assessment': 'Wri

In [143]:
create_df('https://kurser.ku.dk/archive/2017-2018/course/AANA17105U')

ValueError: Length of values does not match length of index